In [1]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/drive/MyDrive/data.zip /content/
!unzip /content/data.zip -d /content/custom_data


Mounted at /content/drive
Archive:  /content/data.zip
   creating: /content/custom_data/labels/
  inflating: /content/custom_data/labels/PDT1000_4.txt  
  inflating: /content/custom_data/labels/PDT1000_6.txt  
  inflating: /content/custom_data/labels/PDT1002_3.txt  
  inflating: /content/custom_data/labels/PDT10039_7.txt  
  inflating: /content/custom_data/labels/PDT10039_8.txt  
  inflating: /content/custom_data/labels/PDT10083_8.txt  
  inflating: /content/custom_data/labels/PDT1011_3.txt  
  inflating: /content/custom_data/labels/PDT10121_8.txt  
  inflating: /content/custom_data/labels/PDT1013_4.txt  
  inflating: /content/custom_data/labels/PDT1013_6.txt  
  inflating: /content/custom_data/labels/PDT10175_3.txt  
  inflating: /content/custom_data/labels/PDT10221_6.txt  
  inflating: /content/custom_data/labels/PDT10232_9.txt  
  inflating: /content/custom_data/labels/PDT10234_15.txt  
  inflating: /content/custom_data/labels/PDT10238_10.txt  
  inflating: /content/custom_data/labe

In [2]:
import os
import shutil
import random

# Reproductibilité
random.seed(42)

# Define source and destination directories
source_data_dir = '/content/custom_data'
dest_data_dir = '/content/data'


# Create the main 'data' directory
os.makedirs(dest_data_dir, exist_ok=True)



# Define the subdirectories for train and validation
splits = {
    'train': 0.7,
    'val': 0.15,
    'test': 0.15
}


# Create the folder structure
for split_name in splits.keys():
    images_path = os.path.join(dest_data_dir, split_name, 'images')
    labels_path = os.path.join(dest_data_dir, split_name, 'labels')
    os.makedirs(images_path, exist_ok=True)
    os.makedirs(labels_path, exist_ok=True)

print(f"Created directory structure under '{dest_data_dir}'")

# Get all image files from the source
all_images = [f for f in os.listdir(os.path.join(source_data_dir, 'images')) if f.endswith(('.bmp'))]
random.shuffle(all_images)

# Calculate split sizes
total_images = len(all_images)
train_count = int(total_images * splits['train'])
val_count = int(total_images * splits['val'])

train_images = all_images[:train_count]
val_images = all_images[train_count:train_count + val_count]
test_images = all_images[train_count + val_count:]


Created directory structure under '/content/data'


In [3]:
import os

VALID_CLASSES = {0,1,2}

def clean_labels(folder):
    fixed = 0
    removed_lines = 0
    emptied_files = 0

    for f in os.listdir(folder):
        if not f.endswith(".txt"):
            continue

        path = os.path.join(folder, f)

        with open(path, "r") as file:
            lines = file.readlines()

        new_lines = []

        for line in lines:
            parts = line.strip().split()
            if len(parts) != 5:
                removed_lines += 1
                continue

            try:
                cls = int(parts[0])
                coords = list(map(float, parts[1:]))
                if cls in VALID_CLASSES and all(0 <= v <= 1 for v in coords):
                    new_lines.append(line)
                else:
                    removed_lines += 1
            except:
                removed_lines += 1

        if len(new_lines) == 0 and len(lines) > 0:
            emptied_files += 1

        if new_lines != lines:
            fixed += 1
            with open(path, "w") as file:
                file.writelines(new_lines)

    print("✔️ Files fixed:", fixed)
    print("🗑️ Removed lines:", removed_lines)
    print("📭 Emptied files:", emptied_files)


print("Cleaning labels...")
clean_labels('/content/custom_data/labels')


Cleaning labels...
✔️ Files fixed: 106
🗑️ Removed lines: 210
📭 Emptied files: 0


In [4]:
print(f"Total images: {total_images}")
print(f"Train images: {train_count}")
print(f"Validation images: {len(val_images)}")
print(f"Test images:{len(test_images)}")


Total images: 1056
Train images: 739
Validation images: 158
Test images:159


In [5]:
# Split the image filenames
# Split images
train_images = all_images[:train_count]
val_images = all_images[train_count:train_count + val_count]
test_images = all_images[train_count + val_count:]


# Function to move files
def move_files(image_list, split_name):
    if not image_list:
        return

    dest_images_path = os.path.join(dest_data_dir, split_name, 'images')
    dest_labels_path = os.path.join(dest_data_dir, split_name, 'labels')

    for img_name in image_list:
        # Move image
        src_img_path = os.path.join(source_data_dir, 'images', img_name)
        shutil.move(src_img_path, dest_images_path)

        # Infer label file name (YOLO format)
        label_name = os.path.splitext(img_name)[0] + '.txt'
        src_label_path = os.path.join(source_data_dir, 'labels', label_name)

        # Move label if it exists
        if os.path.exists(src_label_path):
            shutil.move(src_label_path, dest_labels_path)
        else:
            print(f"⚠️ Warning: Label file '{label_name}' not found for image '{img_name}'.")


# Move files for each split
print(f"Moving {len(train_images)} images to train...")
move_files(train_images, 'train')

print(f"Moving {len(val_images)} images to validation...")
move_files(val_images, 'val')

print(f"Moving {len(test_images)} images to test...")
move_files(test_images, 'test')

print("✅ Dataset splitting and folder structuring complete!")
print(f"Train images: {len(train_images)}")
print(f"Validation images: {len(val_images)}")
print(f"Test images: {len(test_images)}")


Moving 739 images to train...
Moving 158 images to validation...
Moving 159 images to test...
✅ Dataset splitting and folder structuring complete!
Train images: 739
Validation images: 158
Test images: 159


In [6]:
import cv2
import os
import shutil

IMG_DIR = "data/train/images"
LBL_DIR = "data/train/labels"

for img_name in os.listdir(IMG_DIR):

    # Process ONLY bmp images
    if not img_name.lower().endswith(".bmp"):
        continue

    img_path = os.path.join(IMG_DIR, img_name)
    label_name = img_name.replace(".bmp", ".txt")
    lbl_path = os.path.join(LBL_DIR, label_name)

    # Safety check
    if not os.path.exists(lbl_path):
        print(f"⚠️ Label missing for {img_name}, skipped")
        continue

    # Read image
    img = cv2.imread(img_path)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    h, s, v = cv2.split(hsv)

    # HSV enhancement
    s = cv2.add(s, 40)
    v = cv2.add(v, 20)

    hsv_enhanced = cv2.merge([h, s, v])
    enhanced = cv2.cvtColor(hsv_enhanced, cv2.COLOR_HSV2BGR)

    # New names
    new_img_name = img_name.replace(".bmp", "_hsv.bmp")
    new_lbl_name = img_name.replace(".bmp", "_hsv.txt")

    # Save enhanced image
    cv2.imwrite(os.path.join(IMG_DIR, new_img_name), enhanced)

    # Copy label
    shutil.copy(lbl_path, os.path.join(LBL_DIR, new_lbl_name))

print("✅ HSV-enhanced images + labels created successfully")

✅ HSV-enhanced images + labels created successfully


In [7]:
import os

images_dir = "data/train/images"
labels_dir = "data/train/labels"

num_images = len([f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))])
num_labels = len([f for f in os.listdir(labels_dir) if os.path.isfile(os.path.join(labels_dir, f))])

print(f"Number of images in '{images_dir}': {num_images}")
print(f"Number of labels in '{labels_dir}': {num_labels}")

Number of images in 'data/train/images': 1478
Number of labels in 'data/train/labels': 1478


In [8]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.0 MB/s eta 0:00:00


In [9]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'val/images',
      'test': 'test/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

Created config file at /content/data.yaml

File contents:

path: /content/data
train: train/images
val: val/images
test: test/images
nc: 3
names:
- Verdissement
- Endommagement interne
- Morsure et Galerie


In [10]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/data.yaml epochs=100 imgsz=640 batch=16

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kob

In [11]:
!yolo detect val model= /content/runs/detect/train/weights/best.pt data=/content/data.yaml  source=data/test/images split=test

Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 113.1±36.9 MB/s, size: 923.1 KB)
val: Scanning /content/data/test/labels... 159 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 159/159 462.1it/s 0.3s
val: New cache created: /content/data/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 3.4it/s 2.9s
                   all        159        204       0.71      0.686       0.72      0.361
          Verdissement         32         39      0.608      0.513      0.533       0.25
 Endommagement interne         65         75      0.755      0.747      0.801      0.381
    Morsure et Galerie         69         90      0.768        0.8      0.826      0.452
Speed: 2.1ms preprocess, 8.9ms inference, 0.0ms loss, 1.9ms postprocess per image
Results s

In [12]:
%cd /content/
!zip -r runs.zip runs


/content
  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/predict/ (stored 0%)
  adding: runs/detect/train/ (stored 0%)
  adding: runs/detect/train/train_batch8372.jpg (deflated 20%)
  adding: runs/detect/train/BoxF1_curve.png (deflated 10%)
  adding: runs/detect/train/labels.jpg (deflated 31%)
  adding: runs/detect/train/val_batch1_pred.jpg (deflated 23%)
  adding: runs/detect/train/confusion_matrix_normalized.png (deflated 23%)
  adding: runs/detect/train/val_batch0_pred.jpg (deflated 22%)
  adding: runs/detect/train/BoxP_curve.png (deflated 10%)
  adding: runs/detect/train/weights/ (stored 0%)
  adding: runs/detect/train/weights/best.pt (deflated 8%)
  adding: runs/detect/train/weights/last.pt (deflated 8%)
  adding: runs/detect/train/val_batch2_labels.jpg (deflated 22%)
  adding: runs/detect/train/args.yaml (deflated 53%)
  adding: runs/detect/train/val_batch1_labels.jpg (deflated 24%)
  adding: runs/detect/train/train_batch2.jpg (deflated 9%)
  

In [13]:
%cd /content/
!zip -r clean_data.zip data

/content
  adding: data/ (stored 0%)
  adding: data/test/ (stored 0%)
  adding: data/test/images/ (stored 0%)
  adding: data/test/images/PDT5384_6.bmp (deflated 44%)
  adding: data/test/images/PDT3402_11.bmp (deflated 52%)
  adding: data/test/images/PDT9215_1.bmp (deflated 51%)
  adding: data/test/images/PDT1011_3.bmp (deflated 53%)
  adding: data/test/images/PDT9395_7.bmp (deflated 53%)
  adding: data/test/images/PDT5497_8.bmp (deflated 53%)
  adding: data/test/images/PDT20999_8.bmp (deflated 47%)
  adding: data/test/images/PDT1513_5.bmp (deflated 39%)
  adding: data/test/images/PDT939_2.bmp (deflated 52%)
  adding: data/test/images/PDT614_6.bmp (deflated 50%)
  adding: data/test/images/PDT9045_13.bmp (deflated 52%)
  adding: data/test/images/PDT1796_11.bmp (deflated 46%)
  adding: data/test/images/PDT318_6.bmp (deflated 43%)
  adding: data/test/images/PDT7866_16.bmp (deflated 52%)
  adding: data/test/images/PDT20587_20.bmp (deflated 44%)
  adding: data/test/images/PDT1051_5.bmp (defl